# Decision Tree - Diabetes Dataset

In [ ]:
# import Pkg; Pkg.add("CSV")
# import Pkg; Pkg.add("DataFrames")
# import Pkg; Pkg.add("Lathe")
# import Pkg; Pkg.add("DecisionTree")

In [277]:
using CSV, DataFrames, DecisionTree

In [278]:
diabetes = DataFrame(CSV.File("data/diabetes_data.csv"))

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia
,Int64,String7,String3,String3,String3,String3,String3
1,40,Male,No,Yes,No,Yes,No
2,58,Male,No,No,No,Yes,No
3,41,Male,Yes,No,No,Yes,Yes
4,45,Male,No,No,Yes,Yes,Yes
5,60,Male,Yes,Yes,Yes,Yes,Yes
6,55,Male,Yes,Yes,No,Yes,Yes
7,57,Male,Yes,Yes,No,Yes,Yes
8,66,Male,Yes,Yes,Yes,Yes,No
9,67,Male,Yes,Yes,No,Yes,Yes


In [279]:
features = convert(Array, diabetes[:, 1:end-1]);
labels = convert(Array, diabetes[:, end]);

In [280]:
# Training the model
model = build_tree(labels, features)

Decision Tree
Leaves: 32
Depth:  8

In [281]:
# Pruning
# merge leaves having >= 90% combined purity
model = prune_tree(model, 0.9)

Decision Tree
Leaves: 31
Depth:  8

In [282]:
print_tree(model)

Feature 3, Threshold Yes
L-> Feature 2, Threshold Male
    L-> Feature 15, Threshold Yes
        L-> Feature 1, Threshold 37.0
            L-> Feature 14, Threshold Yes
                L-> Feature 1, Threshold 34.5
                    L-> Negative : 5/5
                    R-> Feature 11, Threshold Yes
                        L-> Positive : 6/6
                        R-> Negative : 1/1
                R-> Positive : 6/6
            R-> Positive : 29/29
        R-> Feature 12, Threshold Yes
            L-> Positive : 3/3
            R-> Negative : 12/12
    R-> Feature 4, Threshold Yes
        L-> Feature 11, Threshold Yes
            L-> Feature 6, Threshold Yes
                L-> Negative : 87/88
                R-> Feature 10, Threshold Yes
                    L-> Feature 15, Threshold Yes
                        L-> Negative : 10/10
                        R-> Feature 8, Threshold Yes
                            L-> Positive : 4/4
                            R-> Negative : 3/3
   

In [283]:
nfoldCV_tree(labels, features,4)

2×2 Matrix{Int64}:
 43   4
  2  80

2×2 Matrix{Int64}:
 45   4
  3  77


Fold 1
Classes:  String15["Negative", "Positive"]
Matrix:   
Accuracy: 0.9534883720930233
Kappa:    0.8986645718774549

Fold 2
Classes:  String15["Negative", "Positive"]
Matrix:   
Accuracy: 0.9457364341085271
Kappa:    0.8843641951594314


2×2 Matrix{Int64}:
 52   1
  5  71

2×2 Matrix{Int64}:
 46   3
  1  79


Fold 3
Classes:  String15["Negative", "Positive"]
Matrix:   
Accuracy: 0.9534883720930233
Kappa:    0.9050073637702504

Fold 4
Classes:  String15["Negative", "Positive"]
Matrix:   
Accuracy: 0.9689922480620154
Kappa:    0.933659038313191

Mean Accuracy: 0.9554263565891473


4-element Vector{Float64}:
 0.9534883720930233
 0.9457364341085271
 0.9534883720930233
 0.9689922480620154

In [284]:
test = DataFrame(CSV.File("data/test.csv"))

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia
,Int64,String7,String3,String3,String3,String3,String3
1,48,Female,Yes,Yes,Yes,Yes,Yes
2,58,Female,Yes,Yes,Yes,Yes,Yes
3,32,Female,No,No,No,Yes,No
4,42,Male,No,No,No,No,No


In [285]:
t_features = test[!,1:end-1]
t_labels = test[!,end]

# t_features[1:1,:]
# t_labels[1:1,end]

4-element PooledArrays.PooledVector{String15, UInt32, Vector{UInt32}}:
 "Positive"
 "Positive"
 "Negative"
 "Negative"

In [286]:
arr = convert(Matrix,t_features[3:3,:])

1×16 Matrix{Any}:
 32  "Female"  "No"  "No"  "No"  "Yes"  …  "Yes"  "No"  "No"  "Yes"  "No"

In [288]:
apply_tree(model,arr) == t_labels[3:3,end]

true

# Random Forests

In [289]:
# using 5 random features, 10 trees, and 0.5 portion of samples per tree (optional)
forest = build_forest(labels, features, 5, 10, 0.5)

Ensemble of Decision Trees
Trees:      10
Avg Leaves: 29.8
Avg Depth:  9.4

In [290]:
apply_forest(forest, arr)

1-element Vector{String15}:
 "Negative"

In [291]:
apply_forest_proba(forest, arr, ["Negative", "Positive"])

1×2 Matrix{Float64}:
 0.9  0.1

In [293]:
x  = [1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [294]:
x .+ 2

4-element Vector{Int64}:
 3
 4
 5
 6

In [295]:
x |> sum

10